### Example X: Ray ML Workers

This is a simple example of Ray actors demonstrating that actors are stateful service centers.

We start with a simple program that runs the ResNet50 network to classify 1000 images from ImageNet, one for each class. 
The data is from https://github.com/EliSchwartz/imagenet-sample-images

In [ ]:
import time
import os

from resnet50 import ResNet50

# an object to run the ResNet50 model
srn50 = ResNet50()

# JPEG image files to classify
directory = '../data/imagenet1000'

start_time = time.time()  # Get the current time

# iterate over the sample images
for filename in os.listdir(directory):
    if filename.endswith(".JPEG"):
        try:
            file_path = os.path.join(directory, filename)
            
            # classify the image and return top predicted classes
            preds = srn50.classify_image(file_path)
            print(f"Filename {filename}: predictions {preds}")
        except:
            print(f"Failed to classify. Probably an image error {filename}.")
            pass
            
end_time = time.time()  # Get the current time again

execution_time = end_time - start_time
print("Execution time: ", execution_time, " seconds")

The file [resnet50.py](resnet50.py) shows how simple it is to run computer vision, deep learning models. It loads a pre-trained model  and the parameters needed to normalize input images in the constructor.  The function `classify_image` normalizes the image to a tensor, evaluates the tensor on the model, and then extracts the class names for the top predictions.

This is a serial implementation in that one object runs in a single thread. It could be parallelized in many ways.  We could use `joblib` to create multiple processes. In this case, we are going to use `ray` to build a set of distributed actors. The concept is to instantiate a series of actors each of which has loaded the model. This loading is a one-time cost on instantiation. We can then call remote functions on the actors to classify images. The actors stay around and act as service centers for parallel work.

The Ray implementation in [rayresnet50.py](rayresnet50.py) is the exact same code. It differs only in that it has the `@ray.remote` decorator to indicate that the object will be run as a Ray actor. Most of the complexity lies in the driver code that must launch the remote functions on the actors and complete them asynchronously.

In [2]:
from rayresnet50 import RayResNet50
import ray
import time
import os

num_actors=4

# script to drive parallel program
ray.init(num_cpus=num_actors, ignore_reinit_error=True)

# create the actors and store actor handles
actors = []
for i in range(num_actors):
    actors.append(RayResNet50.remote())

current_actor = 0

directory = '../data/imagenet1000'
files = os.listdir(directory)
roids = [None] * len(files)

start_time = time.time()  # Get the current time

for i in range(len(files)):
    if files[i].endswith(".JPEG"):
        file_path = os.path.join(directory, files[i])
        roids[i] = (actors[i%num_actors].classify_image.remote(file_path))

for i in range(len(files)):
    try:
        if files[i].endswith(".JPEG"):
            preds = ray.get(roids[i])
            print(f"Filename {files[i]}: predictions {preds}")
    except:
        pass

end_time = time.time()  # Get the current time again

execution_time = end_time - start_time
print("Execution time: ", execution_time, " seconds")


2023-11-08 20:27:02,299	INFO util.py:159 -- Outdated packages:
  ipywidgets==7.6.5 found, needs ipywidgets>=8
Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2023-11-08 20:27:04,009	WARNING services.py:1889 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67108864 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=3.36gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2023-11-08 20:27:05,208	INFO worker.py:1642 -- Started a local Ray instance.
(pid=3120) /opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10g

Filename n01440764_tench.JPEG: predictions ['tench', 'barracouta', 'reel']
Filename n01443537_goldfish.JPEG: predictions ['goldfish', 'tench', 'anemone fish']
Filename n01484850_great_white_shark.JPEG: predictions ['great white shark', 'tiger shark', 'gorilla']
Filename n01491361_tiger_shark.JPEG: predictions ['tiger shark', 'great white shark', 'hammerhead']
Filename n01494475_hammerhead.JPEG: predictions ['hammerhead', 'gar', 'swimming trunks']
Filename n01496331_electric_ray.JPEG: predictions ['electric ray', 'stingray', 'clog']
Filename n01498041_stingray.JPEG: predictions ['electric ray', 'stingray', 'flatworm']
Filename n01514668_cock.JPEG: predictions ['drake', 'cock', 'hen']
Filename n01514859_hen.JPEG: predictions ['hen', 'cock', 'vulture']
Filename n01518878_ostrich.JPEG: predictions ['ostrich', 'traffic light', 'water tower']
Filename n01530575_brambling.JPEG: predictions ['brambling', 'junco', 'coucal']
Filename n01531178_goldfinch.JPEG: predictions ['goldfinch', 'indigo bu

2023-11-08 20:27:41,995	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RayResNet50.classify_image() (pid=3121, ip=172.17.0.2, actor_id=9d40affe24789a0522bcbf2d01000000, repr=<rayresnet50.RayResNet50 object at 0xfffe78e5a6b0>)
  File "/home/jupyteruser/ebook/examples/rayresnet50.py", line 55, in classify_image
    img_preds = self.model(self.preprocess(imgtensor).unsqueeze(dim=0))
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_presets.py", line 63, in forward
    img = F.normalize(img, mean=self.mean, std=self.std)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py", line 363, in normalize
    return F_t.normalize(tensor, mean=mean, std=std, inplace=inplace)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_te

Filename n04346328_stupa.JPEG: predictions ['stupa', 'palace', 'castle']
Filename n04347754_submarine.JPEG: predictions ['submarine', 'aircraft carrier', 'liner']
Filename n04350905_suit.JPEG: predictions ['suit', 'Windsor tie', 'groom']
Filename n04355338_sundial.JPEG: predictions ['sundial', 'rule', 'pole']
Filename n04355933_sunglass.JPEG: predictions ['sunglass', 'sunglasses', 'brassiere']
Filename n04357314_sunscreen.JPEG: predictions ['hair spray', 'lotion', 'sunscreen']
Filename n04366367_suspension_bridge.JPEG: predictions ['suspension bridge', 'maillot', 'bikini']
Filename n04367480_swab.JPEG: predictions ['swab', 'laptop', 'broom']
Filename n04370456_sweatshirt.JPEG: predictions ['sweatshirt', 'sunglass', 'chain saw']
Filename n04371430_swimming_trunks.JPEG: predictions ['swimming trunks', 'sunscreen', 'bikini']
Filename n04376876_syringe.JPEG: predictions ['syringe', 'corkscrew', 'screwdriver']
Filename n04380533_table_lamp.JPEG: predictions ['table lamp', 'lampshade', 'spot

2023-11-08 20:27:44,000	ERROR worker.py:405 -- Unhandled error (suppress with 'RAY_IGNORE_UNHANDLED_ERRORS=1'): ray::RayResNet50.classify_image() (pid=3121, ip=172.17.0.2, actor_id=9d40affe24789a0522bcbf2d01000000, repr=<rayresnet50.RayResNet50 object at 0xfffe78e5a6b0>)
  File "/home/jupyteruser/ebook/examples/rayresnet50.py", line 55, in classify_image
    img_preds = self.model(self.preprocess(imgtensor).unsqueeze(dim=0))
  File "/opt/conda/lib/python3.10/site-packages/torch/nn/modules/module.py", line 1130, in _call_impl
    return forward_call(*input, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_presets.py", line 63, in forward
    img = F.normalize(img, mean=self.mean, std=self.std)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py", line 363, in normalize
    return F_t.normalize(tensor, mean=mean, std=std, inplace=inplace)
  File "/opt/conda/lib/python3.10/site-packages/torchvision/transforms/_functional_te

Filename n04523525_vault.JPEG: predictions ['vault', 'monastery', 'prison']
Filename n04525038_velvet.JPEG: predictions ['overskirt', 'trench coat', 'velvet']
Filename n04525305_vending_machine.JPEG: predictions ['vending machine', 'pop bottle', 'jellyfish']
Filename n04532106_vestment.JPEG: predictions ['lab coat', 'vestment', 'apron']
Filename n04536866_violin.JPEG: predictions ['violin', 'cello', 'revolver']
Filename n04540053_volleyball.JPEG: predictions ['volleyball', 'knee pad', 'Walker hound']
Filename n04542943_waffle_iron.JPEG: predictions ['waffle iron', 'mixing bowl', 'measuring cup']
Filename n04548280_wall_clock.JPEG: predictions ['wall clock', 'sliding door', 'wardrobe']
Filename n04548362_wallet.JPEG: predictions ['wallet', 'envelope', 'pillow']
Filename n04550184_wardrobe.JPEG: predictions ['wardrobe', 'medicine chest', 'chiffonier']
Filename n04552348_warplane.JPEG: predictions ['warplane', 'wing', 'airliner']
Filename n04553703_washbasin.JPEG: predictions ['washbasin'